<a href="https://colab.research.google.com/github/Aditi0255/Stock-Trend-Insight-A-News-driven-Stock-Trend-Predictor/blob/main/Stock_Trend_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import re
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [3]:
data=pd.read_csv('/content/Combined_News_DJIA.csv')

FileNotFoundError: ignored

In [ ]:
#Dataset contaning the Top 25 news headlines on the day

data.head()

In [ ]:
data = data.replace(np.nan, ' ', regex=True)
data.isnull().sum().sum()

In [ ]:
data = data.replace('b\"|b\'|\\\\|\\\"', '', regex=True)

In [ ]:
data2=pd.read_csv("/content/upload_DJIA_table.csv")

In [ ]:
#This contains the market Open,High,Low,Close ,Volume and Adj Close
data2.head()

In [ ]:
#Combining the data set with the Market price movement
merge=data.merge(data2,how="inner",on="Date")

In [ ]:
merge.head()

In [ ]:
import sys
!{sys.executable} -m pip install pandas-profiling

In [ ]:
# merge.profile_report()

In [ ]:
headline=[]
for row in range(0,len(merge.index)):
    headline.append(" ".join(str(x) for x in merge.iloc[row,2:27]))

In [ ]:
headline[1]

re.sub(pattern, repl, string, count=0, flags=0)

The re.sub() function stands for a substring and returns a string with replaced values. Multiple elements can be replaced using a list when we use this function.

In [ ]:
clean_headline=[]
for i in range(0,len(headline)):
    clean_headline.append(re.sub("b[(')]",'',headline[i])) #remove b'
    clean_headline[i]=re.sub('b[(")]','',clean_headline[i]) #remove b"
    clean_headline[i]=re.sub("\'",'',clean_headline[i]) #remove \'

Adding the combined and cleaned news back to the merge table

In [ ]:
merge['Combined_News'] = clean_headline

In [ ]:
merge.head()

In [ ]:
#Alternative approach more traditional NLP approach

In [ ]:
samples = list(merge['Combined_News'][:5].values)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split, cross_val_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
token_index = {}  # builds an index of tokens in the data
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1

In [ ]:
max_length = 15

results = np.zeros(shape=(len(samples),   # results will be stored in this array
                          max_length,
                          max(token_index.values()) +1))

In [ ]:
print("Shape of stored results array:", results.shape)
print("Token index of unique words: \n", token_index)

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i,j,index] = 1

In [ ]:
# one hot encoding using keras tokenizer and pad sequencing
X = merge['Combined_News']
encoder = LabelEncoder()
# y = encoder.fit_transform(merge['Combined_News'])
y=merge['Label']
print("shape of input data: ", X.shape)
print("shape of target variable: ", y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
import tqdm
import nltk

import re
from bs4 import BeautifulSoup
import requests
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



In [ ]:


def remove_accented_characters(text):
    text =  unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8','ignore')
    return text



def contractions_text(text):
    return contractions.fix(text)

def stop_words(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []

    for w in word_tokens:

        if w not in stop_words:

            filtered_sentence.append(w)
    return filtered_sentence


In [ ]:
def preprocessor_engine(text):
    corpus =[]
    for sent in tqdm.tqdm(text):
        sent = remove_accented_characters(sent)
        sent = stop_words(sent)
        corpus.append(sent)
    return corpus

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
train_data_pro = preprocessor_engine(X_train)

In [ ]:
test_data_pro = preprocessor_engine(X_test)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
articles_list=[]

In [ ]:
cat_array = train_data_pro # array of news articles text in each category
for i in range(len(cat_array)):
    articles_list.append(cat_array[i])

In [ ]:
wc1 = WordCloud(max_words=1000,
               min_font_size=10,
               height=600,
               width=1600,
               background_color='black',
               contour_color='black',
               colormap='plasma',
               repeat=True,
               stopwords=STOPWORDS)

In [ ]:
plt.figure(figsize=(15,15))

for id in range(len(articles_list)-1):
    plt.subplot(5, 2, id % 10 + 1)
    cloud = wc1.generate(' '.join(articles_list[id]))
    plt.imshow(cloud, interpolation= "bilinear")
    plt.title(f"Wordcloud for news topic {id}")
    plt.axis('off')

In [ ]:
tokenizer = Tokenizer( oov_token='<UNK>')
tokenizer.fit_on_texts(train_data_pro) # build the word index


In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_data_pro)
test_sequences = tokenizer.texts_to_sequences(test_data_pro)

In [ ]:
# pd.Series(train_data_pro).apply(lambda x : len(x.split())).max()

In [ ]:
MAX_SEQUENCE_LENGTH = 424

train_pad_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding='post')
test_pad_sequneces = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding='post')

In [ ]:
total_words=len(tokenizer.word_index)

In [ ]:
from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.layers import Conv1D, SimpleRNN, Bidirectional, MaxPooling1D, GlobalMaxPool1D, LSTM, GRU
from keras.models import Sequential
from keras.regularizers import L1L2

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 300, input_length=MAX_SEQUENCE_LENGTH ))
model.add(Bidirectional(SimpleRNN(64, dropout=0.1, recurrent_dropout=0.20, activation='tanh', return_sequences=True)))
model.add(Bidirectional(SimpleRNN(64, dropout=0.1, recurrent_dropout=0.30, activation='tanh', return_sequences=True)))
model.add(SimpleRNN(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer="rmsprop",loss='binary_crossentropy',
            metrics=['accuracy'])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, SimpleRNN, Dropout, Dense

tokenizer = Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(train_data_pro)
train_sequences = tokenizer.texts_to_sequences(train_data_pro)
test_sequences = tokenizer.texts_to_sequences(test_data_pro)

MAX_SEQUENCE_LENGTH = 424

train_pad_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
test_pad_sequences = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

total_words = len(tokenizer.word_index) + 1  # Add 1 for the padding index

model = Sequential()
model.add(Embedding(total_words, 300, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(SimpleRNN(64, dropout=0.1, recurrent_dropout=0.20, activation='tanh', return_sequences=True)))
model.add(Bidirectional(SimpleRNN(64, dropout=0.1, recurrent_dropout=0.30, activation='tanh', return_sequences=True)))
model.add(SimpleRNN(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Use 'sigmoid' for binary classification
model.summary()

model.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:


history = model.fit(train_pad_sequences , y_train,
                   batch_size=128,
                    epochs=2,
                    validation_split=0.1
                   )

#evalute the model
test_loss, test_acc = model.evaluate(test_pad_sequneces, y_test, verbose=0)
print("test loss and accuracy:", test_loss, test_acc)


# import tensorflow as tf

# train_data = tf.data.Dataset.from_tensor_slices((train_padseq, y_train))
# valid_data = tf.data.Dataset.from_tensor_slices((test_padseq, y_test))

# model.fit(train_data, epochs=10, validation_data=valid_data)

In [ ]:
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def get_polarity(text):
    return TextBlob(text).sentiment.polarity

TextBlob aims to provide access to common text-processing operations through a familiar interface. You can treat TextBlob objects as if they were Python strings that learned how to do Natural Language Processing.



In [ ]:
#Finding the Subjitivity and polarity score from the data set
merge['Subjectivity'] = merge['Combined_News'].apply(get_subjectivity)
merge['Polarity'] = merge['Combined_News'].apply(get_polarity)

In [ ]:
!pip install vaderSentiment


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
!pip install vaderSentiment

Vader Sentiment Analysis helps is putting out the score as overall sentiment score as positive or negative.

In [ ]:
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment= sia.polarity_scores(text)
    return sentiment

In [ ]:
compound=[]
neg=[]
pos=[]
neu=[]
SIA=0

for i in range (0, len(merge['Combined_News'])):
    SIA= getSIA(merge['Combined_News'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    pos.append(SIA['pos'])
    neu.append(SIA['neu'])

In [ ]:
merge['compound']= compound
merge['neg']= neg
merge['pos']=pos
merge['neu']=neu

In [ ]:
df= merge[['Label','Open', 'High', 'Low','Volume','Subjectivity','Polarity','compound','neg','pos','neu']]
df

In [ ]:
X=df
X=np.array(X.drop(['Label'],1))

In [ ]:
y= np.array(df['Label'])

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state= 0)

In [ ]:
model= LinearDiscriminantAnalysis().fit(x_train, y_train)

In [ ]:
predict= model.predict(x_test)

predict

In [ ]:
!pip install scikit-plot



In [ ]:
from scikitplot.metrics import plot_confusion_matrix as plt
plt(y_test,predict)
